In [2]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import random

In [3]:
# Initialize Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id='CLIENT_ID', 
                                                      client_secret='CLIENT_SECRET')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_spotify_track_id(song_name, artist_name):
    # Search for tracks
    results = sp.search(q=f"track:{song_name} artist:{artist_name}", type='track', limit=1)
    
    # Extract track ID if available
    if results['tracks']['items']:
        track_id = results['tracks']['items'][0]['id']
        return track_id
    else:
        return None

In [4]:
# read in song_moods.csv + process the data
# spotify id?
# {mood: songs}
def read_songs(file_path):
    df = pd.read_csv(file_path)
    
    mood_names = df.columns.values.tolist()[2:]
    
    mood_songs = {}
    for i, row in df.iterrows():
        title, artist, moods = row[0], row[1], row[2:]
        spotify_uri = get_spotify_track_id(title, artist)
        
        for mood, val in enumerate(moods):
            if val == 1:
                mood_name = mood_names[mood]
                mood_songs.setdefault(mood_name, []).append({'spotify_uri': spotify_uri, 'title': title, 'artist': artist})
                
    return mood_songs

In [5]:
moods = read_songs('song_moods.csv')

/var/tmp/ipykernel_13139/1874800133.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  title, artist, moods = row[0], row[1], row[2:]


In [6]:
len(moods)

172

In [7]:
# given moods and moods dict
# spotify id?
# return songs based on those moods
def extract_random_songs(mood_dict, moods, num_songs_per_mood=3):
    random_songs = set()

    for mood in moods:
        if mood in mood_dict:
            songs = mood_dict[mood]

            sampled_songs = random.sample(songs, min(num_songs_per_mood, len(songs)))
            random_songs.update('spotify:track:' + song['spotify_uri'] for song in sampled_songs if song['spotify_uri'] is not None)
        else:
            print(f"No songs found for mood: {mood}")

    return list(random_songs)

In [14]:
# given list of songs
# create spotify playlist
# get spotify to recommend songs based on those songs

def create_spotify_playlist(playlist_name, track_uris):
    sp_oauth = SpotifyOAuth(client_id='CLIENT_ID',
                       client_secret='CLIENT_SECRET',
                       redirect_uri='http://localhost:8888/callback',
                       scope='playlist-modify-private')
    
    auth_url = sp_oauth.get_authorize_url()
    print("visit this URL to authorize your account:", auth_url)
    
    auth_response = input("paste the URL you were redirected to here: ")
    
    code = sp_oauth.parse_response_code(auth_response)
    token_info = sp_oauth.get_access_token(code)

    # access token to create a playlist
    sp = spotipy.Spotify(auth=token_info['access_token'])
    
    random_track_uri = random.sample(track_uris, min(5, len(track_uris)))

    playlist = sp.user_playlist_create(sp.me()['id'], playlist_name, public=False)
    recommendations = sp.recommendations(seed_tracks=random_track_uri, limit=30)
    recommended_track_uris = [track['uri'] for track in recommendations['tracks']]
    
    sp.playlist_add_items(playlist['id'], track_uris+recommended_track_uris)
    
    return playlist['id']

In [9]:
songs = extract_random_songs(moods, ['angry', 'angst-ridden', 'brooding', 'confident'])

In [10]:
songs

['spotify:track:5OrUmqP0zk67Nvim1ZXUf1',
 'spotify:track:7AXcgpdL2oJWnI1BCLLeWI',
 'spotify:track:5XuU9htN358NTMCcqRvfDV',
 'spotify:track:3aFRj2jpmzytmMaz9Reade',
 'spotify:track:2nAU8DfyMHGby0DA01vv6L',
 'spotify:track:6GnQ9TEKUjqGnLqmTTpNS2',
 'spotify:track:3NWDb3bbMBTArBblzG4jMX',
 'spotify:track:6UmiroOo97nxwJTYjWBmKE',
 'spotify:track:5eLwwPNN9g5yLrIdyj2ewP',
 'spotify:track:1klrDyGRsCBPofZdqdaZp6',
 'spotify:track:5Q7m1Km199HHMB926xxbrs']

In [15]:
create_spotify_playlist("testing!!! acv", songs)

visit this URL to authorize your account: https://accounts.spotify.com/authorize?client_id=4124d1d1c0b547c889b174b29cc43af7&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=playlist-modify-private


paste the URL you were redirected to here:  http://localhost:8888/callback?code=AQCloyBDE-NiP2w3FFhnHECr6fhKeca0-iN-4CorwTkAXqUVfNCEqFLlr5fP0xZU_Z3AypdlTOUmuwYYNvrXzF7AvIgQxjZYJ5a-43zn5FSKYxdlYt4j24wNwSKALQinp5jyYh13if5K-nAlQMIKAPk66qvNbXa1vCVaWY1-WjtD8Twdw5QZ8OddYd4mvDsxCM4Sy0-Z48v1l5w


/var/tmp/ipykernel_13139/3594710133.py:17: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


'3mHBJH6AVNaCKMU41o0y7d'

In [ ]:
# stretch: personalize based on user listening behavior